In [1]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [2]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [3]:
data_df = pd.read_pickle('cleaned_list_df.pickle')
data_df = data_df.T
data_df

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
0,in the future when apple is at a share your g...,the children were not motivated by achieving ...,tldr do whatever the fuck you want,victorias secret was originally supposed to be...,so was there any posts left after the purge,but i wanted the robot ferret,meme of the month seeding and nominations for ...,redditmc is opening staff positions builders h...,it looks like there may be more to this story ...,decisions regarding mental health care canada...,wait hold the fucking phone he spent like mi...,hi there i ve been looking for a stable job f...,holy mackerel you guys the spotlight is for wr...
1,you ever stare at your account and think wow h...,also proved children like to gamble according ...,so youre telling me people on the internet lie,the ford mustang it was supposed to be a car ...,ok just for some context facebook has an esti...,haha suck it you fools i already got my roboti...,dont have any examples and im not sure if it c...,using this to see if i have new flair,some series get old after a couple updates thi...,public perceptions of exonerees through crime...,trump on harris nasty woman trump on clinton ...,hello people i hope yall are doing good i was ...,congrats ucalamityjeans as tens said i hope ...
2,buy at the top gang 🤡,kind of like how gamers go through rpg games e...,,heels if i remember correctly,they still havent taken down my post about mar...,this not funny rule number oh i forget carry on,certified hood classic,cries in bedrock edition,omg man with the skull cap is the mvp this tim...,does child engagement in household tasks pred...,i could listen to kamala all day every day spe...,company apple i am a bot and this action was...,congratulations it looks like youve been overl...
3,almost responded to an email with thanks retar...,i teach elementary pe to year olds one of th...,this is so funny this subreddit got fucked by ...,is this how family feud gets new questions and...,why does facebook have a fake news problem be...,but the nigerian guy is legit right,arnold schwarzerunner godtier,thank you thats very cash money of you,this update otherwise known as ive had it with...,study on attitudes toward police law enforce...,petty fucking manchild cant stand biden and ...,company netflix i am a bot and this action w...,oh yay ive seen you around on prompts and enj...
4,top trending tickers sentiments and options on...,wouldnt this make lootmystery boxes even worse...,a note about celebrity request bans when riama...,skirts used to be worn in battle by men for mo...,a lot of karens are going to be be saying they...,bruh when i was younger i went to a farm this ...,el dorado explanation,it wont let me connect to the server im using ...,does this mean you owe the man with the skull ...,the relationship between peer support family...,who owns antifacom well it appears it was regi...,company other i am a bot and this action was...,congrats ucalamityjeans as is tradition do ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101136,none,none,none,maybe,none,none,none,none,none,none,none,none,none
101137,none,none,none,is it my cake day i have always wondered how p...,none,none,none,none,none,none,none,none,none
101138,none,none,none,theres a cake emoji next to your username on y...,none,none,none,none,none,none,none,none,none
101139,none,none,none,i think my problem is that im using narwhal ed...,none,none,none,none,none,none,none,none,none


In [4]:
wsb = data_df['wsb'].tolist()
science = data_df['science'].tolist()
ama = data_df['ama'].tolist()
askreddit = data_df['askreddit'].tolist()
worldnews = data_df['worldnews'].tolist()
funny = data_df['funny'].tolist()
dankmemes = data_df['dankmemes'].tolist()
memes = data_df['memes'].tolist()
nosleep = data_df['nosleep'].tolist()
psychology = data_df['psychology'].tolist()
politics = data_df['politics'].tolist()
cscareerquestions = data_df['cscareerquestions'].tolist()
writingprompts = data_df['writingprompts'].tolist()

full_list = wsb + science + ama + askreddit + worldnews + funny + dankmemes + memes + nosleep + psychology + politics + cscareerquestions + writingprompts
full_list

['in the future when apple is at  a share your gran kids will ask why you didnt buy apple at  and youll answer with i bought corn instead',
 'you ever stare at your account and think wow how did i put  into options and think that was safe',
 'buy  at the top gang 🤡',
 'almost responded to an email with thanks retard because  of my social interaction is now through this sub',
 'top trending tickers sentiments and options on rwallstreetbets for the day of august    ────────────────  trending tickers  tickercompanymentionsbulls  bearspricechangepe  tslatesla    spyexchange traded    slvexchange traded    aaplapple    cornexchange traded    rktrocket companies    msftmicrosoft    amdadvanced micro    bathe boeing    fbfacebook    prplpurple innovation    qqqexchange traded    netcloudflare    cscocisco systems    nvdanvidia      ────────────────  trending options  tickerstriketypeexpiration                   ────────────────  trending sentiment  sentimentcomments       ────────────────  ot

In [5]:
vectorizer = TfidfVectorizer(stop_words='english',
                             binary=True, ngram_range=(1,2))

doc_word = vectorizer.fit_transform(wsb)
words = list(np.asarray(vectorizer.get_feature_names())) 
doc_word

<101141x171439 sparse matrix of type '<class 'numpy.float64'>'
	with 439846 stored elements in Compressed Sparse Row format>

In [9]:
doc_word

<101141x171439 sparse matrix of type '<class 'numpy.float64'>'
	with 439846 stored elements in Compressed Sparse Row format>

In [6]:
# cvn = TfidfVectorizer(ngram_range=(1,1))
# data_cvn = cvn.fit_transform(full_list)
# data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# # data_dtmn.columns = data_df.index
# data_dtmn

In [7]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(doc_word))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in vectorizer.vocabulary_.items())
word2id = dict((k, v) for k, v in vectorizer.vocabulary_.items())

In [8]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=75, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 16:51:50: using symmetric alpha at 0.013333333333333334
INFO - 16:51:50: using symmetric eta at 0.013333333333333334
INFO - 16:51:50: using serial LDA version on this node
INFO - 16:51:51: running online LDA training, 75 topics, 50 passes over the supplied corpus of 171439 documents, updating every 30000 documents, evaluating every ~171439 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 16:51:51: training LDA model using 15 processes
INFO - 16:51:51: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/171439, outstanding queue size 1
INFO - 16:51:51: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/171439, outstanding queue size 2
INFO - 16:51:51: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/171439, outstanding queue size 3
INFO - 16:51:51: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/171439, outstanding queue size 4
INFO - 16:51:51: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/17

Process ForkPoolWorker-3:
Process ForkPoolWorker-15:
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    

  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/queues.py", line 96, in get
    with self._rlock:
  File "/home/nick/anaconda3/lib/python3.8/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
KeyboardInterrupt
  File "/home/nick/anaconda3/lib/python3.8/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/home/nick/anaconda3/lib/python3.8/site-packages/gensim/models/ldamodel.py", line 671, in inference
    if len(doc) > 0 and not isinstance(doc[0][0], integer_types):
  File "/home/nick/anaconda3/lib/python3.8/multiprocessing/synchronize.py",

KeyboardInterrupt: 

In [10]:
ldan.save('wsb_idf.model')

INFO - 15:58:10: saving LdaState object under wsb_idf.model.state, separately None
INFO - 15:58:10: storing np array 'sstats' to wsb_idf.model.state.sstats.npy
INFO - 15:58:11: saved wsb_idf.model.state
INFO - 15:58:11: saving LdaMulticore object under wsb_idf.model, separately ['expElogbeta', 'sstats']
INFO - 15:58:11: storing np array 'expElogbeta' to wsb_idf.model.expElogbeta.npy
INFO - 15:58:11: not storing attribute state
INFO - 15:58:11: not storing attribute id2word
INFO - 15:58:11: not storing attribute dispatcher
INFO - 15:58:11: saved wsb_idf.model


In [11]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [12]:
from gensim import corpora
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

## 75 topics, tdfidf, wsb

In [13]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 15:58:32: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 15:58:32: NumExpr defaulting to 8 threads.


## 10 topics, TF-IDF'ed

In [27]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

## 10 topics, CountVectorized

In [20]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 14:13:40: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 14:13:40: NumExpr defaulting to 8 threads.
